In [1]:
%load_ext autoreload
%autoreload 2
# %autoawait asyncio

# Download 

In [2]:
from src.data.download import download_whole_city
from src.settings import DATA_RAW_DIR
from tqdm import tqdm

In [3]:
RESOLUTION = 9

In [4]:
cities = [
    # "Vienna, Austria",  # I had to remove vienna austria as someone removed the city boundary from OSM
    "Minsk, Belarus",
    "Brussels, Belgium",
    "Sofia, Bulgaria",
    "Zagreb, Croatia",
    "Prague, Czech Republic",
    "Tallinn, Estonia",
    "Helsinki, Finland",
    "Paris, France",
    "Berlin, Germany",
    "Reykjavík, Iceland",
    "Dublin, Ireland",
    "Rome, Italy",
    "Nur-Sultan, Kazakhstan",
    "Latvia, Riga",
    "Vilnius, Lithuania",
    "Luxembourg City, Luxembourg",
    "Amsterdam, Netherlands",
    "Oslo, Norway",
    "Warszawa, PL",
    "Kraków, PL",
    "Łódź, PL",
    "Wrocław, PL",
    "Poznań, PL",
    "Gdańsk, PL",
    "Lisbon, Portugal",
    ['Moscow, Russia', 'Zelenogradsky Administrative Okrug', 'Western Administrative Okrug', 'Novomoskovsky Administrative Okrug', 'Troitsky Administrative Okrug'],
    "Belgrade, Serbia",
    "Bratislava, Slovakia",
    "Ljubljana, Slovenia",
    "Madrid, Spain",
    "Stockholm, Sweden",
    "Bern, Switzerland",
    ["London, United Kingdom", "City of London"],
    "New York City, USA",
    "Chicago, USA",
    "San Francisco, USA",
]

In [1]:
for city in tqdm(cities):
    download_whole_city(city, DATA_RAW_DIR)

# Process dataset, select tags, add h3 indices of selected resolution

## Append h3 indices of selected resolution

### Load the Filter (only process what we need)

In [ ]:
# remove tags not of interest
SELECTED_TAGS = [
    "aeroway",
    "amenity",
    "building",
    "healthcare",
    "historic",
    "landuse",
    "leisure",
    "military",
    "natural",
    "office",
    "shop",
    "sport",
    "tourism",
    "water",
    "waterway",
]

problem_columns = [
    'amenity_waste_basket',
    'landuse_grass',
    'historic_tomb',
    'natural_tree',
    'natural_tree_row',
    'natural_valley', # northern Warsaw
]


In [ ]:
from src.data.load_data import load_filter
TAG_FILTER = load_filter("from_wiki.json")

# remove tags that are not in SELECTED TAGS
TAG_FILTER = {
    k: v for k,v in TAG_FILTER.values() if k in SELECTED_TAGS
} 

# remove the problem columns
for tag in problem_columns:
    super_tag, *sub_tag = tag.split("_")
    if isinstance(sub_tag, (list, tuple)):
        sub_tag = "_".join(sub_tag)
    TAG_FILTER[super_tag] = [
        tag for tag in TAG_FILTER[super_tag] if tag != sub_tag
    ]

### Update the City List to Only Include Strings

In [3]:
# update the cities list to only include strings
cities = [city[0] if isinstance(city, (list, tuple)) else city for city in cities]

### Map OSM objects to their Hexagons

In [ ]:
from src.data.make_dataset import add_h3_indices_to_city

In [ ]:
for city in tqdm(cities):
    add_h3_indices_to_city(city, RESOLUTION, filter_values=TAG_FILTER)

## Group selected tags in cities

In [ ]:
from src.data.make_dataset import group_city_tags

In [ ]:
for city in tqdm(cities):
    group_city_tags(city, RESOLUTION, tags=list(TAG_FILTER.keys()), filter_values=TAG_FILTER, fill_missing=True)
    

# Group all cities

This saves a file named `9.feather` to the `DATA_PROCESSED_DIR`

In [ ]:
from src.data.make_dataset import group_cities

df = group_cities(
    cities=cities,
    resolution=RESOLUTION,
    add_city_column=True,
)

